# BOREAL CONTROL CODE 

## Imports 

In [1]:
# from selenium import webdriver
# from selenium.webdriver.common.by import By

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.chrome.service import Service
# from webdriver_manager.chrome import ChromeDriverManager
# from utils import select_aerosol_type
# from utils import select_particle_shape_model
import utils

import os
import time
import math

# Chrome Preferences

In [2]:
download_path = "/Users/dada/Desktop/TFM/EARLINET Database/BCN_27_03_2022"

chrome_options = Options()
prefs = {
    "download.default_directory": download_path,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
}
chrome_options.add_experimental_option("prefs", prefs)

# Useful Paths and Input Parameters

In [3]:
################## PARAMETERS PATHS ##################
xpaths_extinction = {
    'value' : [
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[5]/div/div/div[2]/div[2]/input', # XPath for 355
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[5]/div/div/div[2]/div[3]/input', # XPath for 532 
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[5]/div/div/div[2]/div[4]/input', # XPath for 1064
    ],
    'error' : [
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[5]/div/div/div[3]/div[2]/input',
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[5]/div/div/div[3]/div[3]/input',
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[5]/div/div/div[3]/div[4]/input',
    ]
}
xpaths_backscatter = {
        'value' : [
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[7]/div/div/div[2]/div[2]/input',
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[7]/div/div/div[2]/div[3]/input',
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[7]/div/div/div[2]/div[4]/input',
        ],
        'error' : [
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[7]/div/div/div[3]/div[2]/input',
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[7]/div/div/div[3]/div[3]/input',
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[7]/div/div/div[3]/div[4]/input',           
        ]
}
xpaths_particle_depolarization = {
    'value' : [
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[9]/div/div/div[2]/div[2]/input',
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[9]/div/div/div[2]/div[3]/input',
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[9]/div/div/div[2]/div[4]/input',
    ],
    'error' : [
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[9]/div/div/div[3]/div[2]/input',
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[9]/div/div/div[3]/div[3]/input',
        '/html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[9]/div/div/div[3]/div[4]/input',
    ]
}
################## SELECT WAVELNEGTHS TO BE CONSIDERED ##################
wavelengths_to_select = {
    'extinction' : [1,1,0],
    'backscatter' : [1,1,1],
    'particle depolarization' : [1,1,0],
} 
################## INPUT VALUES ##################
extinction = {
    'value' : [302.26, 300.41],
    'error' : [0.1, 0.1],
}
backscatter = {
    'value' : [7.11, 6.68, 4.3],
    'error' : [0.1, 0.1, 0.1],
}
particle_depolarization = {
    'value' : [0.33, 0.33],
    'error' : [0.1, 0.1],
}

# Main

In [4]:
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://boreal.loa.univ-lille.fr/")

################## SELECT AEROSOL TYPE ##################
utils.select_aerosol_type(driver, 'dust') 
################## SELECT PARTICLE SHAPE MODEL ##################
utils.select_particle_shape_model(driver, 'spheroid') 
################## SELECT WAVELENGTHS ##################
utils.select_wavelengths(driver, wavelengths_to_select)
# ################## INPUT EXTINCTION COEFFICIENTS ##################
utils.input_extinction(driver, extinction)
# ext355 = driver.find_element(By.XPATH, xpaths_extinction['value'][0]).get_attribute('value')
# print(ext355)
# ext532 = driver.find_element(By.XPATH, xpaths_extinction['value'][1]).get_attribute('value')
# print(ext532)
# errext355 = driver.find_element(By.XPATH, xpaths_extinction['error'][0]).get_attribute('value')
# print(errext355)
# errext532 = driver.find_element(By.XPATH, xpaths_extinction['error'][1]).get_attribute('value')
# print(errext532)
all_values_valid = all(utils.check_input_value(driver, xpath) for xpath in xpaths_extinction['value'])
all_errors_valid = all(utils.check_input_value(driver, xpath) for xpath in xpaths_extinction['error'])
if not all_values_valid or not all_errors_valid:
    # Handle the invalid input case here
    print("One or more input values are invalid.")
for xpath in xpaths_extinction['value']:
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    utils.trigger_change_event(driver, element)
# ################## INPUT BACKSCATTER COEFFICIENTS ##################
utils.input_backscatter(driver, backscatter)
# utils.input_extinction(driver, extinction)
# back355 = driver.find_element(By.XPATH, xpaths_backscatter['value'][0]).get_attribute('value')
# print(back355)
# back532 = driver.find_element(By.XPATH, xpaths_backscatter['value'][1]).get_attribute('value')
# print(back532)
# errback355 = driver.find_element(By.XPATH, xpaths_backscatter['error'][0]).get_attribute('value')
# print(errback355)
# errback532 = driver.find_element(By.XPATH, xpaths_backscatter['error'][1]).get_attribute('value')
# print(errback532)
# back1064 = driver.find_element(By.XPATH, xpaths_backscatter['value'][2]).get_attribute('value')
# print(back1064)
# errback1064 = driver.find_element(By.XPATH, xpaths_backscatter['error'][2]).get_attribute('value')
# print(errback1064)
all_values_valid = all(utils.check_input_value(driver, xpath) for xpath in xpaths_backscatter['value'])
all_errors_valid = all(utils.check_input_value(driver, xpath) for xpath in xpaths_backscatter['error'])
if not all_values_valid or not all_errors_valid:
    # Handle the invalid input case here
    print("One or more input values are invalid.")
for xpath in xpaths_backscatter['value']:
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    utils.trigger_change_event(driver, element)
# ################## INPUT PARTICLE DEPOLARIZATION COEFFICIENTS ##################
utils.input_pd(driver, particle_depolarization)
pd355 = driver.find_element(By.XPATH, xpaths_particle_depolarization['value'][0]).get_attribute('value')
print(pd355)
pd532 = driver.find_element(By.XPATH, xpaths_particle_depolarization['value'][1]).get_attribute('value')
print(pd532)
errpd355 = driver.find_element(By.XPATH, xpaths_particle_depolarization['error'][0]).get_attribute('value')
print(errpd355)
errpd532 = driver.find_element(By.XPATH, xpaths_particle_depolarization['error'][1]).get_attribute('value')
print(errpd532)
all_values_valid = all(utils.check_input_value(driver, xpath) for xpath in xpaths_particle_depolarization['value'])
all_errors_valid = all(utils.check_input_value(driver, xpath) for xpath in xpaths_particle_depolarization['error'])
if not all_values_valid or not all_errors_valid:
    # Handle the invalid input case here
    print("One or more input values are invalid.")
for xpath in xpaths_particle_depolarization['value']:
    element = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, xpath)))
    utils.trigger_change_event(driver, element)


submit_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".widget-button")))
driver.execute_script("arguments[0].scrollIntoView(true);", submit_button)
%debug driver.execute_script("arguments[0].click();", submit_button)

time.sleep(60) 
%debug WebDriverWait(driver, 10).until(EC.visibility_of_element_located((By.XPATH, "//pre[contains(text(), 'Finished the inversion at')]")))

output_link = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, "Output results in text format")))
output_link.click()

time.sleep(2)

driver.switch_to.window(driver.window_handles[1])

full_text_content = driver.find_element(By.TAG_NAME,'body').text

# Define the start and end markers
start_marker = "******************************   Retrievals   ******************************"
end_marker = "******************************   Fittings   ******************************"

# Find the start and end
start_index = full_text_content.find(start_marker) + len(start_marker)
end_index = full_text_content.find(end_marker)

# Extract the specific part of the text
specific_text_content = full_text_content[start_index:end_index].strip()

# Save the extracted part to a file
with open('/Users/dada/Desktop/TFM/EARLINET Database/BCN_27_03_2022/Case2845_python.txt', 'w') as file:
    file.write(specific_text_content)

driver.close()
driver.switch_to.window(driver.window_handles[0])


# # driver.quit()

Validation failed for input at /html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[5]/div/div/div[2]/div[4]/input: could not convert string to float: ''
Validation failed for input at /html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[5]/div/div/div[3]/div[4]/input: could not convert string to float: ''
One or more input values are invalid.
0.33
0.33
0.1
0.1
Validation failed for input at /html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[9]/div/div/div[2]/div[4]/input: could not convert string to float: ''
Validation failed for input at /html/body/div/div[2]/div/div/div[6]/div/div/div[2]/div[9]/div/div/div[3]/div[4]/input: could not convert string to float: ''
One or more input values are invalid.
NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(1)<module>()

NOTE: Enter 'c' at the ipdb>  prompt to continue execution.
> <string>(1)<module>()



In [ ]:
submit_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".widget-button")))
submit_button.click()

# time.sleep(20)
wait = WebDriverWait(driver, timeout=300)  # Adjust the timeout as needed
wait.until(EC.visibility_of_element_located((By.XPATH, "//pre[contains(text(), 'Finished the inversion at')]")))

output_link = wait.until(EC.element_to_be_clickable((By.PARTIAL_LINK_TEXT, "Output results in text format")))
output_link.click()

time.sleep(10)

driver.switch_to.window(driver.window_handles[1])

full_text_content = driver.find_element(By.TAG_NAME,'body').text

# Define the start and end markers
start_marker = "******************************   Retrievals   ******************************"
end_marker = "******************************   Fittings   ******************************"

# Find the start and end
start_index = full_text_content.find(start_marker) + len(start_marker)
end_index = full_text_content.find(end_marker)

# Extract the specific part of the text
specific_text_content = full_text_content[start_index:end_index].strip()

# Save the extracted part to a file
with open('/Users/dada/Desktop/TFM/EARLINET Database/BCN_27_03_2022/Case2845_python.txt', 'w') as file:
    file.write(specific_text_content)

driver.close()
driver.switch_to.window(driver.window_handles[0])


driver.quit()